In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
store = pd.read_csv('store.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [71]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [72]:
Seasonal = train.loc[:,['Store','Date','DayOfWeek','Sales','Open','Promo','StateHoliday','SchoolHoliday']]
Seasonal = Seasonal.set_index(['Date']).sort_index()
Seasonal.head()

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,
2013-01-01,1115,2,0,0,0,a,1
2013-01-01,379,2,0,0,0,a,1
2013-01-01,378,2,0,0,0,a,1
2013-01-01,377,2,0,0,0,a,1
2013-01-01,376,2,0,0,0,a,1


In [73]:
Seasonal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 7 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Sales            1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(6), object(1)
memory usage: 62.1+ MB


In [74]:
#Month day year
Seasonal = Seasonal.assign(Month = lambda x: x.index.month).assign(Year = lambda x: x.index.year).assign(Day = lambda x: x.index.day)

In [75]:
#Is december, is july, is february
Seasonal.loc[:,'Is_December'] = 0
Seasonal.loc[Seasonal.Month==12, 'Is_December']=1
Seasonal.loc[:,'Is_july']=0
Seasonal.loc[Seasonal.Month==7, 'Is_july']=1
Seasonal.loc[:,'Is_february']=0
Seasonal.loc[Seasonal.Month==2, 'Is_february']=1

In [76]:
#Is last two weeks of december
Seasonal.loc[:,'Is_last_two_weeks_of_year'] = 0
Seasonal.loc[(Seasonal.Month==12)&(Seasonal.Day>20),'Is_last_two_weeks_of_year']=1

In [77]:
#Is december'14 
Seasonal.loc[:,'Is_december_2014'] =0 
Seasonal.loc[(Seasonal.Month==12)&(Seasonal.Year==2014),'Is_december_2014']=1

In [78]:
#Is monday, Is sunday, Is thursday
Seasonal.loc[:,'Is_monday']=0
Seasonal.loc[(Seasonal.DayOfWeek==1),'Is_monday'] =1
Seasonal.loc[:,'Is_sunday']=0
Seasonal.loc[(Seasonal.DayOfWeek==7),'Is_sunday'] =1
Seasonal.loc[:,'Is_thursday']=0
Seasonal.loc[(Seasonal.DayOfWeek==4),'Is_thursday']=1

In [79]:
#Is_more_promo_sale_store, Is_less_promo_sale_store
Stores_more_promo_sales = [96,  198,  271,  286,  489,  543,  552,  575,  607,  635,  693,  898, 1108]
Stores_less_promo_sales = [85,  259,  262,  274,  299,  353,  423,  512,  530,  562,  676,  733,  769,  948, 1097]
Seasonal.loc[:,'Is_more_promo_sale_store']=0
Seasonal.loc[:,'Is_less_promo_sale_store']=0
Seasonal.loc[Seasonal.Store.isin(Stores_more_promo_sales), 'Is_more_promo_sale_store']=1
Seasonal.loc[Seasonal.Store.isin(Stores_less_promo_sales), 'Is_less_promo_sale_store']=1

In [80]:
#Is_store_greater_public_holiday_sale
Stores_more_public_holiday_sale = [85, 259,  262,  274,  335, 353,  423,  453,  494,  512,  530,  562,  578,  676,  682,
  733,  769,  948, 1081, 1097, 1099]
Seasonal.loc[:,'Is_more_pubHoliday_sale_store']=0
Seasonal.loc[(Seasonal.Store.isin(Stores_more_public_holiday_sale)), 'Is_more_pubHoliday_sale_store'] =1

In [81]:
#Is_store_nonzero_easter_sale
Stores_nonzero_easter_sale = [85,  122,  209,  259,  262,  274,  299,  310,  335,  353,  423,  433,  453,  494,  512,
  524,  530,  562,  578,  676,  682,  732,  733,  769,  863,  867,  931,  948, 1045, 1081, 1097, 1099]
Seasonal.loc[:,'Is_nonzero_easter_sale_store']=0
Seasonal.loc[Seasonal.Store.isin(Stores_nonzero_easter_sale),'Is_nonzero_easter_sale_store']=1

In [82]:
#Is_store_nonzero_christmass_sale
Stores_nonzero_christmass_sale = [85,  259,  262, 274,  299,  335,  353,  423,  494,  512,  530,  562,  676,  682, 733,
  769,  948, 1081, 1097, 1099]
Seasonal.loc[:,'Is_nonzero_christmass_sale_store']=0
Seasonal.loc[Seasonal.Store.isin(Stores_nonzero_christmass_sale),'Is_nonzero_christmass_sale_store']=1

In [83]:
#Is_store_schoolopen_sale_more
Stores_schoolopen_sale_more = [10,   56,   77,   85,  107, 130,  182,  202,  226,  233,  245,  259,  262,  274,  285,
  294,  305,  311,  337,  353,  383,  409,  414,  481,  511,  512,  548,  562,  568,  584, 602,  659,  661,  662,  676, 
708,  733,  744,  769,  773,  807,  818,  868,  937,  948, 970, 1020, 1083, 1097, 1115]

Seasonal.loc[:, 'Is_store_schoolopen_sale_more']=0
Seasonal.loc[Seasonal.Store.isin(Stores_schoolopen_sale_more),'Is_store_schoolopen_sale_more']=1

In [84]:
#Make categorical data from StateHoliday
Seasonal.StateHoliday = Seasonal.StateHoliday.map(lambda x: str(x))
Seasonal = pd.concat([Seasonal, pd.get_dummies(Seasonal.StateHoliday, prefix ='Is_stateholiday')],axis=1)
Seasonal.drop('StateHoliday', axis=1, inplace=True)

In [85]:
Seasonal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 27 columns):
Store                               1017209 non-null int64
DayOfWeek                           1017209 non-null int64
Sales                               1017209 non-null int64
Open                                1017209 non-null int64
Promo                               1017209 non-null int64
SchoolHoliday                       1017209 non-null int64
Month                               1017209 non-null int32
Year                                1017209 non-null int32
Day                                 1017209 non-null int32
Is_December                         1017209 non-null int64
Is_july                             1017209 non-null int64
Is_february                         1017209 non-null int64
Is_last_two_weeks_of_year           1017209 non-null int64
Is_december_2014                    1017209 non-null int64
Is_monday                           1017209 no